# Predictions Demo

This notebook demonstrates end-to-end usage of the prediction utilities:

- Create a `ChemicalReaction` with reactants and solvent
- Fetch products from ASKCOS
- Predict retention times (rtpred.ca)
- Predict lambda max values (chemprop in conda env)



In [ ]:
# Setup imports
from predictions.rxn_classes import ChemicalReaction

# Input reaction
reactants = ["CC(=O)OC(C)=O"]
solvent = "CCO"

# Create reaction
rxn = ChemicalReaction(reactants=reactants, solvents=solvent)


In [ ]:
# Fetch products from ASKCOS (synchronous wrapper)
# Note: This will launch a headless browser and can take time.
products = rxn.fetch_products_from_askcos_sync()



In [ ]:
# Predict retention times for current products (synchronous wrapper)
rxn.predict_products_retention_times_sync()


In [ ]:
# Predict lambda max for products (uses conda run -n uvvismlenv)
rxn.predict_products_lambda_max(conda_env="uvvismlenv")


In [ ]:
# Display results
import pandas as pd

data = [
    {
        "smiles": p.get_smiles(),
        "probability": p.get_probability(),
        "mol_weight": p.get_mol_weight(),
        "retention_time": p.get_retention_time(),
        "lambda_max": p.get_lambda_max(),
    }
    for p in rxn.get_products()
]

pd.DataFrame(data)
